In [ ]:
#!pip install numpy
#!pip install --user -U nltk
#nltk.download('stopwords')
nltk.download('punkt')
#!pip install unidecode
#!pip install matplotlib
#!pip install pyhive

In [ ]:
import pandas as pd
import nltk
import time
import datetime
import boto3
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import RSLPStemmer
from unidecode import unidecode
from pyhive import hive

In [ ]:
#Carrega dados refined do Hive
#df = pd.read_csv("../s3_data/refined/refined.csv", names=["wiki_id", "title", "timestamp", "user","bot"])

conn = hive.Connection(host=host,port= 10000)
dataframe = pd.read_sql("SELECT wiki_id, title, wiki_timestamp, wiki_user, bot FROM defined.wikidata", conn)

In [ ]:
df = pd.read_csv("defined_new.csv", names=["wiki_id", "title", "timestamp", "user","bot"])

In [ ]:
df = pd.read_csv("defined_new.csv", names=["wiki_id", "title", "timestamp", "user","bot"])

In [ ]:
schema, wiki_id, type, namespace, title, comment, wiki_timestamp, wiki_user, bot, minor, patrolled, server_url, server_name, server_script_path, wiki, parsedcomment

## Parte 1
## Pré Processamento

In [ ]:
#Remove linhas com dados nulos, transformando em minusculo e removendo aspas simples
df.dropna(inplace=True)
df["user"] = df["user"].replace({'\'': ''}, regex=True)
lw_text = df["title"].str.lower()
lw_text = lw_text.replace({'\'': ''}, regex=True)
df["user"] = df["user"].str.strip().replace({'^[0-9]*$': 'unknown'}, regex=True)

In [ ]:
#Normalizando timestamp
s = "01/01/2020"
default_timestamp = int(datetime.datetime.strptime('01/01/2020', '%d/%m/%Y').strftime("%s"))
df["timestamp"] = df["timestamp"].str.strip().replace({'^((?![0-9]).)*$': default_timestamp}, regex=True)

In [ ]:
#Substitui valores diferentes de booleano pela item de maior frequencia
max_freq = df.bot.mode()[0]
df["bot"] = df["bot"].str.strip().replace({'^((?!(False|True)).)*$': max_freq.strip()}, regex=True)
#df["bot"] = df["bot"].astype(bool)

In [ ]:
#Cria os tokens dos titulos
tokens =  lw_text.apply(word_tokenize)

In [ ]:
#Normalizando com unicode
tokens_uni = tokens.apply(lambda x: [unidecode(z) for z in x ])

In [ ]:
#Remove stopwords
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords.extend(['categoria','artigos', 'predefinicao','ficheiro','sobre','predefinicoes','redirecionamentos','esbocos','ligados','elemento','inexistentes','ficheiros','usuario','wikidata','paginas','wikipedia','discussao','lista'])
stopwords = set(stopwords + list(punctuation))
title_cleaned = tokens_uni.apply(lambda line:  [w for w in line if not w in stopwords])

In [ ]:
#Cria coluna com os titulos tratados
df["title_cleaned"] = title_cleaned.apply(lambda line: " ".join(line))
df.replace("", np.nan, inplace=True)
df.dropna(inplace=True)
df.head()

In [ ]:
#Cria coluna com os titulos com steamming
def Stemming(sentence):
    stemmer = RSLPStemmer()
    phrase = []
    for word in sentence:
        phrase.append(stemmer.stem(word.lower()))
    return phrase

stemmed_list = title_cleaned.apply(lambda line: Stemming(line))
df["title_stemmed"] = stemmed_list.apply(lambda line: " ".join(line))
df.head()

In [ ]:
nltk.download('wordnet')

In [ ]:
#Cria coluna com os titulos com steamming
from nltk.stem import WordNetLemmatizer
def Lemmatizing(sentence):
    lemmatizer = WordNetLemmatizer()
    phrase = []
    for word in sentence:
        phrase.append(lemmatizer.lemmatize(word.lower()))
    return phrase

lemmatized_list = title_cleaned.apply(lambda line: Lemmatizing(line))
df["title_lemmatized"] = lemmatized_list.apply(lambda line: " ".join(line))
df.head()

## Parte 2

In [ ]:
import pandas as pd
import boto3
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [ ]:
title_bot_true = df[df.bot==True]
title_bot_false = df[df.bot==False]

In [ ]:
# Vec. transform com bots = true
str_list = title_bot_true["title_stemmed"].values
vec = TfidfVectorizer()
vec.fit(str_list)
features_bot_true = vec.transform(str_list)
print(features_bot_true)

In [ ]:
# Vec. transform com bots = false
str_list = title_bot_false["title_stemmed"].values
vec.fit(str_list)
features_bot_false = vec.transform(str_list)
print(features_bot_false)

In [ ]:
labels = 'Bot','Não Bots'
data = [features_bot_true.size, features_bot_false.size]

fig1, ax1 = plt.subplots()

ax1.pie(data, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)

ax1.axis('equal')
ax1.set_title("Bot vs Não Bot")
plt.show()

## Palavras mais populares

In [ ]:
#bot = true
top_N = 10
words = title_bot_true["title_cleaned"].str.cat(sep=' ').split()
rslt = pd.DataFrame(Counter(words).most_common(top_N), columns=['Word', 'Frequency']).set_index('Word')
print(rslt)
rslt.plot.bar(rot=0, figsize=(16,10), width=0.8)

In [ ]:
#bot = false
top_N = 10
words = title_bot_false["title_cleaned"].str.cat(sep=' ').split()
rslt = pd.DataFrame(Counter(words).most_common(top_N), columns=['Word', 'Frequency']).set_index('Word')
print(rslt)
rslt.plot.bar(rot=0, figsize=(16,10), width=0.8)

## K-Means bot = false

In [ ]:
Sum_of_squared_distances = []
K = range(1,30)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(features_bot_false)
    Sum_of_squared_distances.append(km.inertia_)


In [ ]:
#Best result cluster = 150
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
fig = plt.figure(figsize=plt.figaspect(0.5))

ax = fig.add_subplot(1, 2, 1)

#Visualização gráfica 2D     # Converte as features para 2D     
pca = PCA(n_components=2, random_state= 0)
reduced_features = pca.fit_transform(features_bot_false.toarray())

# Converte os centros dos clusters para 2D     
reduced_cluster_centers = pca.transform(km.cluster_centers_)

#Plota gráfico 2D     
ax.scatter(reduced_features[:,0], reduced_features[:,1], c=km.predict(features_bot_false))
ax.scatter(reduced_cluster_centers[:, 0], reduced_cluster_centers[:,1], marker='o', s=150, edgecolor='k')

#Plota números nos clusters     
for i, c in enumerate(reduced_cluster_centers):
    ax.scatter(c[0], c[1], marker='$%d$' % i, alpha=1, s=50, edgecolor='k')

cluster=5
#Adiciona informações no gráfico     
plt.title("Análise de cluster k = %d" % cluster)
plt.xlabel('Dispersão em X')
plt.ylabel('Dispersão em Y')



#Visualização gráfica 3D 

ax = fig.add_subplot(1, 2, 2,projection="3d")

# ax = plt.axes(projection="3d") 
# Adiciona informações no gráfico     
plt.title("Análise de cluster k = %d" % cluster)
plt.xlabel('Dispersão em X')
plt.ylabel('Dispersão em Y')

#converte dados para 3D     
pca = PCA(n_components=3, random_state=0)
reduced_features = pca.fit_transform(features_bot_false.toarray())

#Plota dados em 3D     
ax.scatter3D(reduced_features[:,0], reduced_features[:,1], reduced_features[:,2], marker='o', s=150, edgecolor='k', c=km.predict(features_bot_false))

# Converte os centros dos clusters para 3D     
reduced_cluster_centers = pca.transform(km.cluster_centers_)

#Salva arquivo de imagem 3D     
plt.savefig("imagens/grafico_cluster_k=%d" % cluster)
plt.show()

## K-means bot = True

In [ ]:
Sum_of_squared_distances = []
K = range(1,30)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(features_bot_true)
    Sum_of_squared_distances.append(km.inertia_)

In [ ]:
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
fig = plt.figure(figsize=plt.figaspect(0.5))

ax = fig.add_subplot(1, 2, 1)

#Visualização gráfica 2D     # Converte as features para 2D     
pca = PCA(n_components=2, random_state= 0)
reduced_features = pca.fit_transform(features_bot_true.toarray())

# Converte os centros dos clusters para 2D     
reduced_cluster_centers = pca.transform(km.cluster_centers_)

#Plota gráfico 2D     
ax.scatter(reduced_features[:,0], reduced_features[:,1], c=km.predict(features_bot_true))
ax.scatter(reduced_cluster_centers[:, 0], reduced_cluster_centers[:,1], marker='o', s=150, edgecolor='k')

#Plota números nos clusters     
for i, c in enumerate(reduced_cluster_centers):
    ax.scatter(c[0], c[1], marker='$%d$' % i, alpha=1, s=50, edgecolor='k')

cluster=5
#Adiciona informações no gráfico     
plt.title("Análise de cluster k = %d" % cluster)
plt.xlabel('Dispersão em X')
plt.ylabel('Dispersão em Y')



#Visualização gráfica 3D 

ax = fig.add_subplot(1, 2, 2,projection="3d")

# ax = plt.axes(projection="3d") 
# Adiciona informações no gráfico     
plt.title("Análise de cluster k = %d" % cluster)
plt.xlabel('Dispersão em X')
plt.ylabel('Dispersão em Y')

#converte dados para 3D     
pca = PCA(n_components=3, random_state=0)
reduced_features = pca.fit_transform(features_bot_true.toarray())

#Plota dados em 3D     
ax.scatter3D(reduced_features[:,0], reduced_features[:,1], reduced_features[:,2], marker='o', s=150, edgecolor='k', c=km.predict(features_bot_true))

# Converte os centros dos clusters para 3D     
reduced_cluster_centers = pca.transform(km.cluster_centers_)

#Salva arquivo de imagem 3D     
plt.savefig("imagens/grafico_cluster_k=%d" % cluster)
plt.show()